In [4]:
import time
import glob as glob
from zipfile import ZipFile
from bs4 import BeautifulSoup
import dateutil.parser as dparser
from datetime import datetime
import pandas as pd
import multiprocessing as mp
import numpy as np

In [5]:
# root_dir = '/media/shane/cloud-availability-sacheen-2020-05-11/atlassian' # one directory
# root_dir = '/media/shane/cloud-availability-sacheen-2020-05-11/atlassian/2020/20203' # subset of above
root_dir = '/media/shane/cloud-availability-sacheen-2020-05-11/atlassian*' # all directories

In [6]:
ts = int("1585223263")

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

2020-03-26 11:47:43


In [7]:
def extract_html_data(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    result = []
    
    timestamp = soup.find('meta', attrs={'name':'issued'}).get('content')
    ts_uct = datetime.utcfromtimestamp(int(timestamp)).strftime('%Y-%m-%d %H:%M:%S')
    result.append(ts_uct)
    
    for status in soup.find_all("div", {"class": "component-inner-container"}):
        result.append(' '.join(status.text.split()))
    
#     # status of the service in last 24hrs (no problems, possible problems, problems)
#     for status in soup.find_all("div", {"class": "alert"}):
#         result.append(' '.join(status.text.split()))
    
#     # time since problems started (empty unless problems are ongoing)
#     for problems_since in soup.find_all("div", {"class": "event"}):
#         try:
#             status = ' '.join(problems_since.text.split())
#             date = str(dparser.parse(status, fuzzy=True))
#             result.append([date.split()[0].split('-')[1:], date.split()[1]])
#         except:
#             # some months are apparently out of range
#             result.append('')
    
#     # TECHNICAL DEBT: problems_since is not appended if tag does not exist in the html_doc
#     if len(result) != 5:
#         result.append('')
    
#     # most reported problems at this time
#     mrp = []
#     for most_reported in soup.find_all("li"):
#         if '%' in most_reported.text:
#             mrp.append(' '.join(most_reported.text.split()))
#     result.append(mrp)

    return result

In [8]:
def read_file(file):
    source = file.split('/')[4]
    archive = ZipFile(file, 'r')
    namelist = archive.namelist()
    ret = []
    for item in namelist:
        if 'html' in item:
            html = archive.read(item)
            service = item.split('/')[1].split('.')[0]
            data = extract_html_data(html)
            data.insert(0, service)
            data.insert(0, source)
            ret.append(data)
    archive.close()
    return(ret)

In [9]:
files = glob.glob(root_dir + '/**/*.zip', recursive=True)

pool = mp.Pool(mp.cpu_count())

start_time = time.time()
results = pool.map(read_file, [file for file in files])
pool.close()
end_time = time.time()

In [10]:
print(end_time - start_time)

200.6598403453827


In [11]:
res = []
for outer in results:
    for inner in outer:
        res.append(inner)
print(len(res))

23864


In [13]:
# column_names = ['source','service','timestamp','status','problems_since','most_reported_problems']
df = pd.DataFrame(res) #columns=column_names)
df.head(50)

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,atlassian,confluence,2020-05-08 15:00:09,View Content ? Operational,Create and Edit Operational,Comments Operational,Authentication and User Management Operational,Search Operational,Administration ? Operational,Notifications Operational,...,None,None,None,None,None,None,None,None,None,None
1,atlassian,jira-software,2020-05-08 15:00:08,Viewing content ? Operational,Create and edit ? Operational,Authentication and User Management Operational,Search ? Operational,Notifications ? Operational,Administration ? Operational,Marketplace ? Operational,...,None,None,None,None,None,None,None,None,None,None
2,atlassian,jira-core,2020-05-08 15:00:10,Viewing content ? Operational,Create and edit ? Operational,Authentication and User Management Operational,Search ? Operational,Notifications ? Operational,Administration ? Operational,Marketplace ? Operational,...,None,None,None,None,None,None,None,None,None,None
3,atlassian,jira-align,2020-05-08 15:00:26,Pod 1 Operational,Pod 2 Operational,Pod 3 Operational,Pod 4 Operational,Pod 5 Operational,Pod 6 Operational,Pod 7 Operational,...,None,None,None,None,None,None,None,None,None,None
4,atlassian,bitbucket,2020-05-08 14:46:49,Website ? Operational,API ? Operational,SSH ? Operational,Authentication and user management Operational,Git via HTTPS ? Operational,Mercurial via HTTPS ? Operational,Webhooks ? Operational,...,None,None,None,None,None,None,None,None,None,None
5,atlassian,support,2020-05-08 14:00:07,Support Portal Operational 90 days ago 100.0 %...,Ticketing Operational 90 days ago 100.0 % upti...,Knowledge Base Operational 90 days ago 100.0 %...,Community Operational 90 days ago 100.0 % upti...,API Docs Operational 90 days ago 100.0 % uptim...,Public Issues Tracker ? Operational 90 days ag...,Training ? Operational 90 days ago 100.0 % upt...,...,None,None,None,None,None,None,None,None,None,None
6,atlassian,developers,2020-05-08 14:35:53,APIs Operational 90 days ago 100.0 % uptime Today,Bitbucket Cloud APIs Operational 90 days ago 1...,Confluence Cloud APIs Operational 90 days ago ...,Jira Cloud APIs Operational 90 days ago 100.0 ...,User APIs Operational 90 days ago 100.0 % upti...,Webhooks Operational 90 days ago 100.0 % uptim...,Developer Operational 90 days ago 99.99 % upti...,...,None,None,None,None,None,None,None,None,None,None
7,atlassian,statuspage,2020-05-08 15:00:11,Hosted Pages Operational 90 days ago 99.98 % u...,HTTP Pages Operational 90 days ago 99.96 % upt...,HTTPS Pages Operational 90 days ago 99.96 % up...,Status Embed Widget Operational 90 days ago 10...,Public API Operational 90 days ago 99.99 % upt...,Shortlinks Operational 90 days ago 100.0 % upt...,Management Operational 90 days ago 100.0 % upt...,...,Librato Integration Operational,New Relic Integration Operational,Datadog Integration Operational,Custom Integration Operational,Support Sites Operational,Ticketing Operational,Knowledge Base Operational,API Documentation Operational,Metastatuspage ? Operational,Marketing Site (www.statuspage.io) Operational
8,atlassian,trello,2020-05-08 14:44:34,Trello.com ? Operational 90 days ago 99.97 % u...,API Operational 90 days ago 99.98 % uptime Today,help.trello.com ? Operational,Support Message Email Processing ? Operational,Support Message Web App ? Operational,None,None,...,None,None,None,None,None,None,None,None,None,None
9,atlassian,partners,2020-05-08 15:00:21,Partner Portal ? Operational 90 days ago 100.0...,Partner Support Portal ? Operational 90 days a...,Partner Directory Operational 90 days ago 100....,Partner Business Center Operational 90 days ag...,Enablement Academy Operational 90 days ago 100...,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
df.to_csv(r'/home/shane/Documents/thesis/output/atlassian.csv', index=False)